In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Library

In [ ]:
pip install tensorflow.io

In [ ]:
from IPython.display import Audio

import tensorflow as tf
import tensorflow_io as tfio
import matplotlib.pyplot as plt
import numpy as np
import glob

In [ ]:
#pip install python_speech_features

In [ ]:
import os
import random
import pickle
import pandas as pd
import seaborn as sns

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#from python_speech_features import mfcc

##load data

In [ ]:
daftar_dangdut = glob.glob('/content/drive/MyDrive/Dataset/Dangdut/lagu/*')
#daftar_pop = glob.glob('/content/drive/MyDrive/Dataset/POP/lagu/*')
#daftar_jpop = glob.glob('/content/drive/MyDrive/Dataset/JPOP/lagu/*')
daftar_kpop = glob.glob('/content/drive/MyDrive/Dataset/KPOP/lagu/*')

In [ ]:
#baca file dan masukin ke list
dangdut =[]
for i in range(len(daftar_dangdut)):
  lagu = tf.io.read_file(daftar_dangdut[i])
  suara = tfio.audio.decode_mp3(lagu)
  dangdut.append(suara)
'''
pop =[]
for i in range(len(daftar_pop)):
  lagu = tf.io.read_file(daftar_pop[i])
  suara = tfio.audio.decode_mp3(lagu)
  pop.append(suara)

jpop =[]
for i in range(len(daftar_jpop)):
  lagu = tf.io.read_file(daftar_jpop[i])
  suara = tfio.audio.decode_mp3(lagu)
  jpop.append(suara)
'''
kpop =[]
for i in range(len(daftar_kpop)):
  lagu = tf.io.read_file(daftar_kpop[i])
  suara = tfio.audio.decode_mp3(lagu)
  kpop.append(suara)

In [ ]:
#train
x=[]
y=[]

#mencacah/motong lagu
for i in range(len(daftar_dangdut)):
  for j in range(0, len(dangdut[i])-44100, 11025):
    x.append(dangdut[i][j:j+44100])
    y.append(0)

for i in range(len(daftar_kpop)):
  for j in range(0, len(kpop[i])-44100, 11025):
    x.append(kpop[i][j:j+44100])
    y.append(1)

'''
for i in range(len(daftar_pop)):
  for j in range(0, len(pop[i])-44100, 11025):
    x.append(pop[i][j:j+44100])
    y.append(2)

for i in range(len(daftar_jpop)):
  for j in range(0, len(jpop[i])-44100, 11025):
    x.append(jpop[i][j:j+44100])
    y.append(3)'''

'\nfor i in range(len(daftar_pop)):\n  for j in range(0, len(pop[i])-44100, 11025):\n    x.append(pop[i][j:j+44100])\n    y.append(2)\n\nfor i in range(len(daftar_jpop)):\n  for j in range(0, len(jpop[i])-44100, 11025):\n    x.append(jpop[i][j:j+44100])\n    y.append(3)'

In [ ]:
x = np.asarray(x)
y = np.asarray(y)

In [ ]:
label_encoder = LabelEncoder()
label_encoded = label_encoder.fit_transform(y)

In [ ]:
label_encoded=label_encoded[:,np.newaxis]

In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(label_encoded)

In [ ]:
y = one_hot_encoded

In [ ]:
maks = np.max(x)
mins = np.min(x)

#normalisasi
x = (x-mins)/(maks-mins)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
y_test = one_hot_encoder.inverse_transform(y_test)

In [ ]:
y_test

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [ ]:
print(x.shape) 
print(x_train.shape) 
print(y_train.shape) 
print(x_test.shape) 
print(y_test.shape) 

(8744, 44100, 2)
(6995, 44100, 2)
(6995, 2)
(1749, 44100, 2)
(1749, 1)


## **Model**

In [ ]:
# CNN 1D
from keras.models import Sequential
from keras.layers import Conv1D, Flatten, Dense, Activation


model = Sequential()

model.add(Conv1D(64, kernel_size=3, input_shape = (x.shape[1],2), activation = 'relu'))
model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(units = 2, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train, batch_size=25, epochs=50)

Epoch 1/50
280/280 [==============================] - 138s 456ms/step - loss: 0.7972 - accuracy: 0.5118
Epoch 2/50
280/280 [==============================] - 127s 455ms/step - loss: 0.6962 - accuracy: 0.4988
Epoch 3/50
280/280 [==============================] - 127s 455ms/step - loss: 0.7073 - accuracy: 0.5084
Epoch 4/50
280/280 [==============================] - 127s 454ms/step - loss: 0.6971 - accuracy: 0.4968
Epoch 5/50
280/280 [==============================] - 127s 454ms/step - loss: 0.6932 - accuracy: 0.5032
Epoch 6/50
280/280 [==============================] - 127s 455ms/step - loss: 0.6917 - accuracy: 0.5217
Epoch 7/50
280/280 [==============================] - 128s 456ms/step - loss: 0.6932 - accuracy: 0.5066
Epoch 8/50
280/280 [==============================] - 127s 454ms/step - loss: 0.6932 - accuracy: 0.5066
Epoch 9/50
280/280 [==============================] - 128s 456ms/step - loss: 0.6931 - accuracy: 0.5066
Epoch 10/50
280/280 [==============================] - 127s 454m

In [ ]:
#model.save_weights('f2_songclassifier.h5')
#read file
model.load_weights('/content/drive/MyDrive/Dataset/f2_songclassifier.h5')

In [ ]:
pred = model.predict(x_test)

In [ ]:
pred

array([[9.9988478e-01, 1.4364719e-04],
       [1.2555247e-09, 1.0000000e+00],
       [9.9059433e-01, 8.7026656e-03],
       ...,
       [9.9996573e-01, 5.4185966e-05],
       [9.9994558e-01, 7.5622942e-05],
       [5.1286817e-04, 9.9960506e-01]], dtype=float32)

In [ ]:
pred[50]

array([9.995847e-01, 4.634261e-04], dtype=float32)

In [ ]:
predict = np.argmax(pred, axis=1)
predict

array([0, 1, 0, ..., 0, 0, 1])

In [ ]:
y_test

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [ ]:
import pandas as pd

df=pd.DataFrame(predict,columns=['predict'])
df['y_test'] = y_test
df

,predict,y_test
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
...,...,...
1744,0,0
1745,0,0
1746,0,0
1747,0,0


In [ ]:
accuracy_score(predict, y_test)

0.9439679817038308

###random play song

In [ ]:
d = random.choice(daftar_kpop)
print(d)
Audio(d)

Output hidden; open in https://colab.research.google.com to view.

##Predict w/ new file

In [ ]:
ftest = '/content/drive/MyDrive/Kuli-AH/smt5/PowPow 💪/ML/TUBES - Dataset/Dataset 1 dan 2 (Gumaman + Lagu Asli)/KPOP/Lagu/BTOB - 그리워하다 (Missing You).mp3'
ftest

'/content/drive/MyDrive/Kuli-AH/smt5/PowPow 💪/ML/TUBES - Dataset/Dataset 1 dan 2 (Gumaman + Lagu Asli)/KPOP/Lagu/BTOB - 그리워하다 (Missing You).mp3'

In [ ]:
f_Test = tf.io.read_file(ftest)
s_Test = tfio.audio.decode_mp3(f_Test)

In [ ]:
input = []
for i in range(0, len(s_Test)-44100, 11025):
    input.append(s_Test[i:i+44100])

input = np.asarray(input)
maks = np.max(input)
min = np.min(input)

input = (input-min)/(maks-min)

In [ ]:
hasil = model.predict(input)
hasil = np.argmax(hasil,axis=1)
hasil

array([1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,

In [ ]:
hasil = pd.DataFrame(hasil)
jmlh_0 = hasil[(hasil[0]==0)].shape[0]
jmlh_1 = hasil[(hasil[0]==1)].shape[0]

if jmlh_0 > jmlh_1 :
  print('dangdut')
  d = random.choice(daftar_dangdut)
  print(d)
elif jmlh_0 < jmlh_1 :
  print('kpop')
  d = random.choice(daftar_kpop)
  print(d)
else :
  print('not known')

Audio(d)

Output hidden; open in https://colab.research.google.com to view.

##Flask

In [ ]:
!pip install flask

In [ ]:
!pip install flask-ngrok

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   
              

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:13 http://ppa.launch

In [ ]:
! ngrok authtoken 23N1IcF96hHTFMCLqhpdFkFEo8J_cs4VCtJuHfxbkXDNzyYT

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!cp -r "/content/drive/MyDrive/Dataset/templates" .
!cp -r "/content/drive/MyDrive/Dataset/static" .

In [ ]:
# flask_ngrok_example.py
from flask import Flask, render_template, request, redirect, url_for
from flask_ngrok import run_with_ngrok

ALLOWED_EXTENSIONS = set(['mp3'])

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def hello():
    return render_template("index.html")

@app.route("/fungsi2", methods=['POST','GET'])
def fungsi2():
  if request.method == 'POST':
    audio = request.files['audio']
    audio.save('test.mp3')
    f_test = tf.io.read_file('test.mp3')
    s_test = tfio.audio.decode_mp3(f_test)

    input = []
    for i in range(0, len(s_test)-44100, 11025):
        input.append(s_test[i:i+44100])
    input = np.asarray(input)
    maks = np.max(input)
    min = np.min(input)
    input = (input-min)/(maks-min)

    hasil = model.predict(input)
    hasil = np.argmax(hasil,axis=1)
    hasil = pd.DataFrame(hasil)
    jmlh_0 = hasil[(hasil[0]==0)].shape[0]
    jmlh_1 = hasil[(hasil[0]==1)].shape[0]

    if jmlh_0 > jmlh_1 :
      result = 'The genre of that song is Dangdut'
      d = random.choice(daftar_dangdut)
    elif jmlh_0 < jmlh_1 :
      result = 'The genre of that song is KPOP'
      d = random.choice(daftar_kpop)
    else :
      result = 'not known'
    
    p = Audio(d)
    return render_template("fungsi2.html",result="{}".format(result),
                            audio="{}".format(audio),
                            d="{}".format(d),
                            p="{}".format(p))

  else :
    return render_template("fungsi2.html")

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6f5b-34-86-117-36.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Jan/2022 08:20:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:38] "GET /static/css/bootstrap.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:38] "GET /static/css/base.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:39] "GET /static/css/magnific-popup.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:39] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:39] "GET /static/css/flexslider.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:39] "GET /static/css/fonts.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:39] "GET /static/js/jquery-1.12.4.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:43] "GET /static/js/jquery.flexslider-min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:43] "GET /static/js/smooth-scroll.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:43] "GET /static/js/jquery.magnific-popup.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2022 08:20:46] "GET /fungsi2 HTTP/1.1" 200 -
127.0.0.1